<a href="https://colab.research.google.com/github/IamJac/MachineLearningAlgorithms/blob/main/HousePricesPredictions02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UPLOADING DATA**

In [1]:
from google.colab import files
file1=files.upload()
file2=files.upload()
print("Data Successfuly loaded")

Saving train.csv to train.csv


Saving test.csv to test.csv
Data Successfuly loaded


**CONVERTING THE DATA INTO A DATAFRAME**

In [2]:
import pandas as pd
df1=pd.read_csv("train.csv")#Training data
df2=pd.read_csv("test.csv")#Testing data
print(df1.head())
print(df2.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

**EXPLORATORY DATA ANALYSIS**

In [3]:
#Exploring the testing dataframe
print(df2.info())
print(df2.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [4]:
#Exploring the training dataset
print(df1.info())
print(df1.isnull().sum())
print(df1.corr)
print(df1.shape)
print(df1.describe())
print(df1.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

**DATA PREPROCESSING**

In [113]:
train_data=df1.drop(["SalePrice"],axis=1)#This is data from train.csv
all_y_training=df1.SalePrice#This is the SalePrice column from train.csv
test_data=df2#This if data from test.csv
numerical_columns=train_data.select_dtypes(include=["int64","float64"]).columns
categorical_columns=train_data.select_dtypes(include=["object"]).columns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
numerical_transformer=Pipeline(steps=[
    ("imputer",SimpleImputer(strategy="mean")),
     ('scaler',RobustScaler())
])
categorical_transformer=Pipeline(steps=[
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("onehot",OneHotEncoder(handle_unknown="ignore"))
])
preprocessor=ColumnTransformer(transformers=[
    ("num",numerical_transformer,numerical_columns),
    ("cat",categorical_transformer,categorical_columns)
])
train_x,val_x,train_y,val_y=train_test_split(train_data,df1.SalePrice,test_size=0.20,random_state=0)
print(train_x.shape)
print(val_x.shape)
print(train_y.shape)
print(val_y.shape)
print()
print("SUCCESSFUL PREPROCESSING")


(1168, 80)
(292, 80)
(1168,)
(292,)

SUCCESSFUL PREPROCESSING


**MODEL CREATION AND FITTING**

In [114]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,root_mean_squared_error,r2_score,mean_absolute_percentage_error
model=XGBRegressor(n_estimators=1000,learning_rate=0.05,max_depth=6,gamma=0.5,colsample_bytree=0.5,reg_alpha=0.5,reg_lambda=0.5,min_child_weight=3,eval_metric='mape')
my_pipeline=Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("model",model)
])
my_pipeline.fit(train_x,train_y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', '...
                              feature_types=None, gamma=0.5, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None, min_child_weight=3,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=1000,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [117]:

preds_on_validation_set=my_pipeline.predict(val_x)
print("MODEL PERFFORMANCE ON THE VALIDATION DATA SET")
print("R2_score",r2_score(val_y,preds_on_validation_set))
print("Mean Absolute Error",mean_absolute_error(val_y,preds_on_validation_set))
print("Mean Absolute Percentage Error",mean_absolute_percentage_error(val_y,preds_on_validation_set))

MODEL PERFFORMANCE ON THE VALIDATION DATA SET
R2_score 0.887677013874054
Mean Absolute Error 15774.359602418665
Mean Absolute Percentage Error 0.08894197296917263


Testing the model on the whole training data

In [116]:
preds_on_train_set=my_pipeline.predict(train_data)
print("MODEL PERFFORMANCE ON THE TRAINING DATA SET")
print("R2_score",r2_score(all_y_training,preds_on_train_set))
print("Mean Absolute Error",mean_absolute_error(all_y_training,preds_on_train_set))
print("Mean Absolute Percentage Error",mean_absolute_percentage_error(all_y_training,preds_on_train_set))
comp_data=pd.DataFrame({"Actual":all_y_training,"Predicted":preds_on_train_set})
print(comp_data.head())


R2_score 0.9753686785697937
Mean Absolute Error 3423.1991197559933
Mean Absolute Percentage Error 0.019528116177703815
   Actual      Predicted
0  208500  208816.718750
1  181500  174725.765625
2  223500  213920.796875
3  140000  140109.000000
4  250000  286085.343750


MAKING PREDICTIONS USING THE TEST DATA

In [66]:
preds_on_test_set=my_pipeline.predict(test_data)
print(preds_on_test_set)

[124497.84  156809.86  181213.34  ... 161179.84  114087.875 230081.77 ]


**SAVING PREDICTIONS IN CSV FORMAT**

In [120]:
submission=pd.DataFrame({"Id":test_data.Id,"SalePrice":preds_on_test_set})
submission.to_csv("submission.csv",index=False)
print("Conversion Successful")

Conversion Successful


**SAVING CSV FILE IN COMPUTER MEMORY**

In [121]:
files.download("submission.csv")
print("SUCCESSFUL")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SUCCESSFUL
